# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
from dotenv import load_dotenv 
import os 
load_dotenv()

PRICE_DATA = os.getenv('PRICE_DATA')

print(PRICE_DATA)

../../05_src/data/prices/


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

['../../05_src/data/prices/CTAS/CTAS_2008.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2009.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2012.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2002.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2024.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2013.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2020.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2021.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2007.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2017.parquet',
 '../../05_s

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_rets = (dd_px.groupby('ticker', group_keys=False)
    .apply(lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag=x['Adj_Close'].shift(1),
        returns=(x['Adj_Close'] / x['Adj_Close'].shift(1)) - 1,
        hi_lo_range=x['High'] - x['Low']
    ))
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [25]:
# Write your code below.

df_rets = dd_rets.compute()
df_rets['rolling_avg_return'] = df_rets.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

I don't believe so. I think it would've been faster to do so using dask, as the calculations can be made in parallel by my understanding. It could then be turned into a pandas df after that. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.